**Importar bibliotecas**

In [1]:
import os, flopy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import flopy.utils.binaryfile as bf
import shutil

flopy is installed in C:\Users\Jerson\miniconda3\lib\site-packages\flopy


**MODELO DE FLUJO**

**Asignación de nombre de modelo**

In [2]:
model_ws='../4_OUT/R_SIM'
modelname='Modelo_Flujo'
exe_name= '../1_SOLVER/mf2005.exe'
mf = flopy.modflow.Modflow(modelname, exe_name=exe_name, model_ws=model_ws)

**Discretización Espacial**

In [3]:
nlay = 30
nrow = 1
ncol = 100
delr=10.
delc=1.
top = 510.
botm = np.ones((nlay), dtype=np.int32)
for i in range (nlay):
    botm[i]= 500 -(i*10)
nper = 3
nstp = [1]+[1]*2 
perlen = [1] + [864000]*1 + [2523743999]*1
steady = [True] + [False]*2

dis=flopy.modflow.ModflowDis(mf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, 
                             botm=botm, nper=nper, perlen=perlen, steady=steady, itmuni=1, lenuni=2)

In [4]:
#botm
#dis.plot()

**Definición de celdas activas/paquete BAS**



In [5]:
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[1:,:,0] = -1
ibound[21:,:,-1] = -1
#ibound

**Altura inicial - strt** 

In [6]:
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=top)
#bas.plot()

**Nivel constante - CHD**

In [7]:
chdspd={}
chdspd[0]=[[i, 0, 0, 500., 500.] for i in range(1,30)]
for i in range (21,30):
    chdspd[0].append(([i, 0, 99, 300.,300.])) 
chdspd[1]=[[i, 0, 0, 500., 500.] for i in range(1,30)]
for i in range (21,30):
    chdspd[1].append(([i, 0, 99, 300.,300.])) 
chdspd[2]=[[i, 0, 0, 500., 500.] for i in range(1,30)]
for i in range (21,30):
    chdspd[2].append(([i, 0, 99, 300.,300.])) 
    
chd = flopy.modflow.ModflowChd(mf, stress_period_data=chdspd)

**Conductividad Hidráulica**

In [8]:
kx=pd.read_excel('../3_IN/SIMULACION_K.xlsx', sheet_name=None, header=None) 
#kx

**Paquete solucionador mf2005**

In [9]:
pcg =flopy.modflow.ModflowPcg(mf, mxiter=85, iter1=57, npcond=1, nbpol=1, hclose=1e-05, rclose=1e-05, relax=1,
                              iprpcg=0, mutpcg=0, ihcofadd=0)

In [10]:
#numero de simulaciones estocasticas
n_estocasticas=['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19',
                '20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39',
                '40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59',
                '60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79',
                '80','81','82','83','84','85','86','87','88','89','90','91','92','93','94','95','96','97','98','99']

In [11]:
for i in n_estocasticas:
    hk=kx[i].to_numpy()
    
    lpf=flopy.modflow.ModflowLpf(mf, laytyp=1, ipakcb=53, hk=hk, vka=hk, hdry=-2e+20, chani=-1.0, ss=1e-05, sy=0.15)
                                 #laywet=1, wetfct=1, iwetit=4, ihdwet=0)
    
    #**Output Control - Paquete OC** 

    spd_oc = {}
    for kper in range(nper):
        for kstp in range(nstp[kper]):
            spd_oc[(kper, kstp)] = ['save head','save budget']
    
    oc= flopy.modflow.ModflowOc(mf,stress_period_data=spd_oc, compact=True)
        
    #**LMT Linkage with MT3DMS for multi-species mass transport modeling**
    lmt = flopy.modflow.ModflowLmt(mf, output_file_name='mt3d.ftl')
    
    #**Escribir los archivos de salida mf2005**
    mf.write_input()
    
    #**Correr Modelo de Flujo**
    mf.run_model(silent=True)
    
    #Copiar y Guardar archivos de simulaciones
    shutil.copy(model_ws +'/'+'Modelo_flujo.hds', model_ws+'/'+'hds'+'/'+"Modelo_Flujo_{}.hds".format(i))
    shutil.copy(model_ws +'/'+'Modelo_flujo.cbc', model_ws+'/'+'cbc'+'/'+"Modelo_Flujo_{}.cbc".format(i))
    shutil.copy(model_ws +'/'+'Modelo_flujo.list', model_ws+'/'+'list'+'/'+"Modelo_Flujo_{}.list".format(i))
    shutil.copy(model_ws +'/'+'mt3d.ftl', model_ws+'/'+'mt3d'+'/'+"mt3d_{}.ftl".format(i))

**MODELO DE TRANSPORTE**

**Definicion del modelo**

In [12]:
for j in n_estocasticas:
    ftlfilename='mt3d'+'/'+'/mt3d_{}.ftl'.format(j)
    namemt3d='transModel'
    mt_model = flopy.mt3d.Mt3dms(modelname=namemt3d,  model_ws=model_ws, ftlfilename=ftlfilename, version='mt3d-usgs', 
                                 exe_name='../1_SOLVER/mt3d-usgs_1.1.0_64.exe', modflowmodel=mf)
          
    #BTN file
          
    timprs=[10800., 1728000., 4320000., 7776000., 15552000., 31536000., 94672800., 157788000., 220903200., 946728000., 
            1105000000., 1578000000.]
    icbund = np.ones((nlay, nrow, ncol), dtype=np.int32)
    icbund[1:,:,0] = -1
    nprs=len(timprs)
    nper = 3
    perlen = [1] + [864000]*1 + [2523800000]*1
    btn = flopy.mt3d.Mt3dBtn(mt_model, ncomp=1, nper=nper, mcomp=1, tunit='sec', lunit='m', munit='g', prsity=0.25,
                             sconc=0.0, cinact=-1e+30, thkmin=0.01, ifmtcn=12, ifmtnp=5, ifmtrf=12, ifmtdp=12, 
                             perlen=perlen, savucn=True, nprs=nprs, timprs=timprs, dt0=0, mxstrn=50000,ttsmult=1., 
                             ttsmax=0, icbund=icbund)
    
    #Paquete de Advencion - ADV
          
    adv = flopy.mt3d.Mt3dAdv(mt_model, mixelm=3, percel=1, mxpart=75000, nadvfd=2, itrack=2, wd=1, dceps=1e-05, 
                        nplane=0, npl=0, nph=10, npmin=2, npmax=20, npsink=10, dchmoc=0.01)
          
    #Solucionador - GCG
    
    gcg = flopy.mt3d.Mt3dGcg(mt_model, mxiter=1, iter1=200, isolve=3, ncrs=0, cclose=1e-06, iprgcg=1)
    
    #Paquete de Dispersion - DSP
    
    dsp = flopy.mt3d.Mt3dDsp(mt_model, al=0.0, multiDiff=True, dmcoef=0, trpt=0.1, trpv=0.01)
    
    itype = flopy.mt3d.Mt3dSsm.itype_dict()
    #itype
    
    #Concentracion del contamiante - SSM
    
    ssm_data = {}
    ssm_data[1] = [(k, 0, 0, 100, 1) for k in range (1,30)]
    for k in range (1,30):
        ssm_data[1].append((k, 0, 0, 100, -1)) 
    ssm_data[2] = [(k, 0, 0, 0, 1) for k in range (1,30)]
    for k in range (1,30):
        ssm_data[2].append((k, 0, 0, 0, -1)) 
    ssm = flopy.mt3d.Mt3dSsm(mt_model, stress_period_data=ssm_data)
    
    #Escribir archivos de Salida del modelo de transporte - MT3D
    
    mt_model.write_input()
          
    #Correr de modelo de Transporte - MT3D
    
    mt_model.run_model(silent=True)
    
    shutil.copy(model_ws +'/'+'MT3D001.UCN', model_ws+'/'+'ucn'+'/'+"MT3D001_{}.UCN".format(j))
    shutil.copy(model_ws +'/'+'MT3D001.MAS', model_ws+'/'+'t_mas'+'/'+"MT3D001_{}.MAS".format(j))
    shutil.copy(model_ws +'/'+'transModel.list', model_ws+'/'+'t_list'+'/'+"Trans_{}.list".format(j))